## 2. Trial by trial decoding analysis

### (Map Ripple's decode, insert into the Database.)
[This notebook uses TrialChoice() and RippleTimes() to populate the TrialChoiceReplay() table.]
##### All the tables mentioned above are under shijiegu github/spyglass/shijiegu/Analysis_SGU.py. It is not in the Franklab spyglass.

Jun 23, 2024
Shijie Gu

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import spyglass as nd
import pandas as pd
import os
# ignore datajoint+jupyter async warnings
import warnings
warnings.simplefilter('ignore', category=DeprecationWarning)
warnings.simplefilter('ignore', category=ResourceWarning)

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import logging
import multiprocessing

FORMAT = '%(asctime)s %(message)s'

logging.basicConfig(level='INFO', format=FORMAT, datefmt='%d-%b-%y %H:%M:%S')

from spyglass.common import (Session, IntervalList,IntervalPositionInfo,
                             LabMember, LabTeam, Raw, Session, Nwbfile,
                            Electrode,LFPBand,interval_list_intersect)
from spyglass.common.common_interval import _intersection

import spyglass.spikesorting as ss
from spyglass.spikesorting.v0 import (SortGroup, 
                                    SortInterval,
                                    SpikeSortingPreprocessingParameters,
                                    SpikeSortingRecording, 
                                    SpikeSorterParameters,
                                    SpikeSortingRecordingSelection,
                                    ArtifactDetectionParameters, ArtifactDetectionSelection,
                                    ArtifactRemovedIntervalList, ArtifactDetection,
                                      SpikeSortingSelection, SpikeSorting,
                                   CuratedSpikeSortingSelection,CuratedSpikeSorting,Curation)
#from spyglass.decoding.clusterless import (UnitMarks,
#                                           UnitMarkParameters,UnitMarksIndicatorSelection,
#                                          UnitMarksIndicator)
from spyglass.utils.nwb_helper_fn import get_nwb_copy_filename
from spyglass.common.common_position import IntervalPositionInfo, IntervalPositionInfoSelection
from spyglass.common.common_position import IntervalLinearizedPosition

from spyglass.shijiegu.load import load_epoch_data
# Here are the analysis tables specific to Shijie Gu
from spyglass.shijiegu.Analysis_SGU import (TrialChoice,
                                   TrialChoiceReplay,
                                   RippleTimes,
                                   DecodeResultsLinear,get_linearization_map,
                                   find_ripple_times,mua_thresholder,sort_replays,
                                   classify_ripples,classify_ripple_content,
                                   load_everything,MUA,
                                   TetrodeNumber
                                  )
from spyglass.shijiegu.Analysis_SGU import EpochPos,RippleTimesWithDecode,ExtendedRippleTimes,ExtendedTrialChoiceReplay,TrialChoiceReplayTransition
from spyglass.shijiegu.ripple_add_replay import (replay_parser_master,plot_ripples_replays)

[2025-06-14 18:34:22,155][INFO]: DataJoint 0.14.4 connected to shijiegu-alt@lmf-db.cin.ucsf.edu:3306
14-Jun-25 18:34:22 DataJoint 0.14.4 connected to shijiegu-alt@lmf-db.cin.ucsf.edu:3306


In [13]:
# the only cell to be edited
nwb_file_name_all = ['klein20231111.nwb']
nwb_file_name = nwb_file_name_all[0]
nwb_copy_file_name = get_nwb_copy_filename(nwb_file_name_all[0])

In [14]:
DecodeResultsLinear & {'nwb_file_name': nwb_copy_file_name}

nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,classifier_param_name a name for this set of parameters,encoding_set a name for this set of encoding,posterior posterior within that interval (1D)
klein20231111_.nwb,02_Rev2Session1,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,=BLOB=
klein20231111_.nwb,04_Rev2Session2,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,=BLOB=
klein20231111_.nwb,06_Rev2Session3,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,=BLOB=
klein20231111_.nwb,08_Rev2Session4,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,=BLOB=


In [6]:
RippleTimes & {'nwb_file_name': nwb_copy_file_name}

nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,ripple_times ripple times within that interval,ripple_times_04sd ripple times within that interval,ripple_times_1sd ripple times within that interval,ripple_times_2sd ripple times within that interval


In [16]:
TrialChoice() & {'nwb_file_name':nwb_copy_file_name}

nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),"epoch_name session name, get from IntervalList","choice_reward pandas dataframe, choice"
klein20231111_.nwb,2,02_Rev2Session1,=BLOB=
klein20231111_.nwb,4,04_Rev2Session2,=BLOB=
klein20231111_.nwb,6,06_Rev2Session3,=BLOB=
klein20231111_.nwb,8,08_Rev2Session4,=BLOB=
klein20231111_.nwb,10,10_Rev2Session5,=BLOB=


## 1. For one epoch

In [8]:
epoch_num = 2

In [9]:
(epoch_name,log_df,ripple_times,decode,
     head_speed,head_orientation,linear_position_df,
     lfp_df,theta_df,ripple_df,neural_df,mua_df) = replay_parser_master(nwb_file_name,epoch_num,
                                                                       encoding_set = '2Dheadspeed_above_4')

[19:55:08][WARNING] Spyglass: Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.
18-May-25 19:55:08 Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.


epoch name 02_Seq2Session1
epoch name 02_Seq2Session1
epoch_pos_name pos 1 valid times



KeyboardInterrupt



In [29]:
log_df

,timestamp_H,Home,timestamp_O,OuterWellIndex,rewardNum,ripple_H,ripple_O,replay_H,replay_O,ripple_ID_H,ripple_ID_O,current,future_H,future_O,past,past_reward
1,1.704665e+09,1.0,1.704665e+09,1.0,2.0,[],"[[[1704665025.3625145, 1704665025.5325146]]]",[],[[[1]]],[],[0],1.0,1.0,3.0,NaN,NaN
2,1.704665e+09,1.0,1.704665e+09,3.0,1.0,[],[],[],[],[],[],3.0,3.0,2.0,1.0,1.0
3,1.704665e+09,1.0,1.704665e+09,2.0,2.0,[],[],[],[],[],[],2.0,2.0,4.0,3.0,1.0
4,1.704665e+09,1.0,1.704665e+09,4.0,2.0,[],[],[],[],[],[],4.0,4.0,1.0,2.0,2.0
5,1.704665e+09,1.0,1.704665e+09,1.0,1.0,"[[[1704665107.002473, 1704665107.106473], [170...",[],"[[[], []]]",[],[1],[],1.0,1.0,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,1.704667e+09,1.0,1.704667e+09,4.0,2.0,"[[[1704666606.3097098, 1704666606.4297097], [1...",[],"[[[], [], []]]",[],[73],[],4.0,4.0,1.0,3.0,2.0
79,1.704667e+09,1.0,1.704667e+09,1.0,1.0,[],[],[],[],[],[],1.0,1.0,2.0,4.0,4.0
80,1.704667e+09,1.0,1.704667e+09,2.0,1.0,"[[[1704666645.6816895, 1704666645.7816896]]]",[],[[[]]],[],[74],[],2.0,2.0,3.0,1.0,4.0
81,1.704667e+09,1.0,1.704667e+09,3.0,2.0,[],[],[],[],[],[],3.0,3.0,NaN,2.0,4.0


### 2. Visual Inspection

In [26]:
mua_thresh = (MUA & {'nwb_file_name' : nwb_copy_file_name,'interval_list_name':epoch_name}).fetch1('mean')

## find tetrodes with signal
groups_with_cell=(SpikeSortingRecordingSelection & {
        'nwb_file_name' : nwb_copy_file_name}).fetch('sort_group_id')
groups_with_cell=np.setdiff1d(groups_with_cell,[100,101])
channel_IDs = list(neural_df.keys())
    
spikeColInd = {}
for g in groups_with_cell:
    spikeColInd_ = np.argwhere(np.isin(channel_IDs,(Electrode() &  {'nwb_file_name' : nwb_copy_file_name,
                                                     'electrode_group_name':str(g)}).fetch('electrode_id'))).ravel()
    spikeColInd[g] = spikeColInd_

# Pre-make result directory:
decoding_path=(DecodeResultsLinear() & {'nwb_file_name':nwb_copy_file_name}).fetch('posterior')[0]
exampledir=os.path.join(os.path.split(decoding_path)[0],nwb_file_name,'SWR',epoch_name)
isExist = os.path.exists(exampledir)
if not isExist:
    os.makedirs(exampledir)
savefolder = exampledir
print(savefolder)

for t in log_df.index[:-2]:
    # Home
    ripples = log_df.loc[t].ripple_H
    ripple_table_inds = log_df.loc[t].ripple_ID_H

    if not np.isnan(log_df.loc[t].timestamp_H):
        plottimes = [log_df.loc[t].timestamp_H,log_df.loc[t].timestamp_H + 6]
        savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name + '_whole_trial'
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                        linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                        neural_df,mua_df,head_speed,head_orientation,
                        savename,savefolder,wholeTrial = 1,plottimes=plottimes,mua_thresh = mua_thresh)
            
    savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name
    plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                    linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                    neural_df,mua_df,head_speed,head_orientation,
                    savename,savefolder,wholeTrial = 0,mua_thresh = mua_thresh)

/cumulus/shijie/recording_pilot/lewis/decoding/lewis20240107.nwb/SWR/04_Rev2Session2


In [43]:
%debug

> /home/shijiegu/Documents/spyglass/src/spyglass/shijiegu/ripple_add_replay.py(202)add_trial()
    200     for r_ind in ripple_times.index:
    201         (t0,t1) = (ripple_times.loc[r_ind,'start_time'],ripple_times.loc[r_ind,'end_time'])
--> 202         linear_position_subset = linear_position_df.isel(time=np.logical_and(linear_position_df.time >= t0,
    203                                                                                         linear_position_df.time <= t1))
    204         linear_position = np.array(np.mean(linear_position_subset.linear_position))



ipdb>  exit


### 3. for all sessions

#### without plot

In [62]:
for nwb_file_name in nwb_file_name_all:
    nwb_copy_file_name = get_nwb_copy_filename(nwb_file_name)
    all_epochs=list((TrialChoice() & {'nwb_file_name':nwb_copy_file_name}).fetch('epoch'))
    for epoch_num in all_epochs:
        (epoch_name,log_df,ripple_times,decode,
         head_speed,head_orientation,linear_position_df,
         lfp_df,theta_df,ripple_df,neural_df,mua_df) = replay_parser_master(nwb_file_name,epoch_num,
                                                                            encoding_set = '2Dheadspeed_above_4_andlowmua',#'2Dheadspeed_above_4',
                                                                            decode_threshold_method = 'MUA_0SD',
                                                                            extended = False)

[15:43:37][WARNING] Spyglass: Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.
12-Apr-25 15:43:37 Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.


epoch name 02_Seq2Session1
epoch name 02_Seq2Session1
epoch_pos_name pos 1 valid times


DataJointError: fetch1 should only return one tuple. 0 tuples found

In [63]:
DecodeResultsLinear & {'nwb_file_name': 'eliot20221016_.nwb'}

nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,classifier_param_name a name for this set of parameters,encoding_set a name for this set of encoding,posterior posterior within that interval (1D)


#### with plot

In [ ]:
%matplotlib agg
for nwb_file_name in nwb_file_name_all:
    nwb_copy_file_name = get_nwb_copy_filename(nwb_file_name)
    all_epochs=list((TrialChoice() & {'nwb_file_name':nwb_copy_file_name}).fetch('epoch'))
    for epoch_num in all_epochs:
        (epoch_name,log_df,ripple_times,decode,
         head_speed,head_orientation,linear_position_df,
         lfp_df,theta_df,ripple_df,neural_df,mua_df) = replay_parser_master(nwb_file_name,epoch_num,
                                                                            encoding_set = '2Dheadspeed_above_4',#'2Dheadspeed_above_4',
                                                                            decode_threshold_method = 'MUA_0SD',
                                                                            extended = False)

    """
    REAL WORK ENDS HERE, THE REST IS JUST PLOTTING
    """

    ## find tetrodes with signal
    groups_with_cell=(SpikeSortingRecordingSelection & {
        'nwb_file_name' : nwb_copy_file_name}).fetch('sort_group_id')
    groups_with_cell=np.setdiff1d(groups_with_cell,[100,101])
    channel_IDs = list(neural_df.keys())
    
    spikeColInd = {}
    for g in groups_with_cell:
        spikeColInd_ = np.argwhere(np.isin(channel_IDs,(Electrode() &  {'nwb_file_name' : nwb_copy_file_name,
                                                     'electrode_group_name':str(g)}).fetch('electrode_id'))).ravel()
        spikeColInd[g] = spikeColInd_

    # Pre-make result directory:
    decoding_path=(DecodeResultsLinear() & {'nwb_file_name':nwb_copy_file_name}).fetch('posterior')[0]
    exampledir = os.path.join(os.path.split(decoding_path)[0],nwb_file_name,'SWR',epoch_name,'statespace')
    isExist = os.path.exists(exampledir)
    if not isExist:
        os.makedirs(exampledir)
    savefolder = exampledir
    print(savefolder)

    mua_thresh = (MUA & {'nwb_file_name' : nwb_copy_file_name,'interval_list_name':epoch_name}).fetch1('mean')


    for t in log_df.index[:-2]:
        # Home
        ripples = log_df.loc[t].ripple_H
        ripple_table_inds = log_df.loc[t].ripple_ID_H
        
        if not np.isnan(log_df.loc[t].timestamp_H):
            plottimes = [] #[log_df.loc[t].timestamp_H,log_df.loc[t].timestamp_H + 6]
            
            savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name + '_whole_trial'+'_statespace'
            plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                         linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                         neural_df,mua_df,head_speed,head_orientation,
                         savename,savefolder,wholeTrial = 1,plottimes=plottimes,mua_thresh = mua_thresh)
            
        savename = 'trial' + str(t) + '_home_' + nwb_copy_file_name[:-5] + '_' + epoch_name +'_statespace'
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                     linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                     neural_df,mua_df,head_speed,head_orientation,
                     savename,savefolder,wholeTrial = 0,mua_thresh = mua_thresh)
    
        
            
        # Outer Well
        ripples = log_df.loc[t].ripple_O
        ripple_table_inds = log_df.loc[t].ripple_ID_O

        plottimes = [log_df.loc[t].timestamp_O,log_df.loc[t].timestamp_O + 6]
        savename = 'trial' + str(t) + '_well_' + nwb_copy_file_name[:-5] + '_' + epoch_name + '_whole_trial'+'_statespace'
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                     linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                     neural_df,mua_df,head_speed,head_orientation,
                     savename,savefolder,wholeTrial = 1,plottimes=plottimes,mua_thresh = mua_thresh)
        
        savename = 'trial' + str(t) + '_well_' + nwb_copy_file_name[:-5] + '_' + epoch_name +'_statespace'
        plot_ripples_replays(ripple_times,ripples,ripple_table_inds,
                     linear_position_df,decode,spikeColInd,lfp_df,theta_df,
                     neural_df,mua_df,head_speed,head_orientation,
                     savename,savefolder,wholeTrial = 0,mua_thresh = mua_thresh)
    
        

[08:46:51][WARNING] Spyglass: Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.
15-Jun-25 08:46:51 Deprecation: this class has been moved out of spyglass.common.common_position
	TrackGraph -> spyglass.linearization.v0.main.TrackGraph
Please use the new location.
[2025-06-15 08:46:51,677][WARNING]: Skipped checksum for file with hash: 02e28a13-9803-1b1e-a0c8-0e8260a98aa9, and path: /stelmo/nwb/analysis/klein20231111/klein20231111_18TK4QPZ4X.nwb
15-Jun-25 08:46:51 Skipped checksum for file with hash: 02e28a13-9803-1b1e-a0c8-0e8260a98aa9, and path: /stelmo/nwb/analysis/klein20231111/klein20231111_18TK4QPZ4X.nwb


epoch name 02_Rev2Session1
epoch name 02_Rev2Session1
epoch_pos_name pos 1 valid times


[2025-06-15 08:46:51,696][WARNING]: Skipped checksum for file with hash: 02e28a13-9803-1b1e-a0c8-0e8260a98aa9, and path: /stelmo/nwb/analysis/klein20231111/klein20231111_18TK4QPZ4X.nwb
15-Jun-25 08:46:51 Skipped checksum for file with hash: 02e28a13-9803-1b1e-a0c8-0e8260a98aa9, and path: /stelmo/nwb/analysis/klein20231111/klein20231111_18TK4QPZ4X.nwb
[2025-06-15 08:46:51,702][WARNING]: Skipped checksum for file with hash: 02e28a13-9803-1b1e-a0c8-0e8260a98aa9, and path: /stelmo/nwb/analysis/klein20231111/klein20231111_18TK4QPZ4X.nwb
15-Jun-25 08:46:51 Skipped checksum for file with hash: 02e28a13-9803-1b1e-a0c8-0e8260a98aa9, and path: /stelmo/nwb/analysis/klein20231111/klein20231111_18TK4QPZ4X.nwb
[2025-06-15 08:47:08,862][WARNING]: Skipped checksum for file with hash: 3efe4415-f71e-c0cd-9445-3090ec4143dc, and path: /stelmo/nwb/analysis/klein20231111/klein20231111_0TQEWI2JX0.nwb
15-Jun-25 08:47:08 Skipped checksum for file with hash: 3efe4415-f71e-c0cd-9445-3090ec4143dc, and path: /stel

In [17]:
RippleTimesWithDecode & {'nwb_file_name': nwb_copy_file_name}

nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,classifier_param_name a name for this set of parameters,encoding_set a name for this set of encoding,decode_threshold_method a name for this thresholding method,ripple_times ripple times within that interval
klein20231111_.nwb,02_Rev2Session1,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_0SD,=BLOB=
klein20231111_.nwb,04_Rev2Session2,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_0SD,=BLOB=
klein20231111_.nwb,06_Rev2Session3,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_0SD,=BLOB=
klein20231111_.nwb,08_Rev2Session4,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_0SD,=BLOB=
klein20231111_.nwb,10_Rev2Session5,default_decoding_gpu_4armMaze,2Dheadspeed_above_4,MUA_0SD,=BLOB=


In [63]:
%debug

> /home/shijiegu/Documents/spyglass/src/spyglass/shijiegu/ripple_add_replay.py(191)add_trial()
    189     for r_ind in ripple_times.index:
    190         (t0,t1) = (ripple_times.loc[r_ind,'start_time'],ripple_times.loc[r_ind,'end_time'])
--> 191         trial_number = np.argwhere(log_df.timestamp_H >= t0).ravel()[0]
    192         # this way only works if the trial is legal.
    193         if np.isin(trial_number,legal_trials):



ipdb>  t0


1704570633.7066205


ipdb>  t1


1704570633.841621


...

ipdb>  np.array(log_df.timestamp_H)[-2]


nan


ipdb>  np.array(log_df.timestamp_O)[-2]


1704570641.6037498


ipdb>  exit


In [7]:
((TetrodeNumber() & {'nwb_file_name': nwb_copy_file_name,"epoch_name":"02_Rev2Session1"})).fetch1("ca1_tetrode_ind")

array([  0,  12,  14,  17,  25,  31,  33,  54,  62,  73,  79,  89,  93,
        98, 102, 108, 112, 115, 118, 126, 130, 139, 142, 146, 152, 157])

## Ends here